</br>

# 5. 常微分方程式の解を用いた熱水分収支式

状微分方程式の解を用いた計算では、  
- 熱・水分流計算  
- 熱・水分収支計算  

これらが一括で行われる。従って計算結果で得られる値は温度および水分化学ポテンシャルの2変数である。  

以下の関数基本的な使い方としては、  

関数名：
- cal_newvalue_by_ODE

引数（キーワード引数）：
- cell_cen : 熱水分収支計算を行うcellを入力する（Porous_material_based_miu内のCell構造体を必ず入力すること）。
- cell_mns : cell_cenに対してmns側に位置するcellを入力する（Cell構造体もしくはAir_based_RH内のAir構造体を入力すること）。
- cell_pls : cell_cenに対してpls側に位置するcellを入力する（Cell構造体もしくはAir_based_RH内のAir構造体を入力すること）。
- dt       : 計算の時間刻みを入力する。
- nx       : 重力に対する応答方向を入力する。水平方向の計算なら0、垂直方法（plsが上方向）なら1を入力する（デフォルト値として0を採用している）。
- heat_gain     : （主に境界条件用）日射熱取得が存在する場合、値を入力する（デフォルト値は0）。
- moisture_gain : （主に境界条件用）降雨など水分取得が存在する場合、値を入力する（デフォルト値は0）。

戻り値（複数）：
- 実数Float64（温度）, 実数Float64（水分化学ポテンシャル）

#### 物性値の平均化に関する関数

In [41]:
lam(cell_mns::Cell, cell_pls::Cell)  = balance_equation_by_ODE.sum_resistance( val_mns = lam(cell_mns),  val_pls = lam(cell_pls),  len_mns = dx2(cell_mns), len_pls = dx2(cell_pls) )
ldmg(cell_mns::Cell, cell_pls::Cell) = balance_equation_by_ODE.sum_resistance( val_mns = ldmg(cell_mns), val_pls = ldmg(cell_pls), len_mns = dx2(cell_mns), len_pls = dx2(cell_pls) )
ldtg(cell_mns::Cell, cell_pls::Cell) = balance_equation_by_ODE.sum_resistance( val_mns = ldtg(cell_mns), val_pls = ldtg(cell_pls), len_mns = dx2(cell_mns), len_pls = dx2(cell_pls) )
ldm(cell_mns::Cell, cell_pls::Cell)  = balance_equation_by_ODE.sum_resistance( val_mns = ldml(cell_mns) + ldmg(cell_mns), val_pls = ldml(cell_pls) + ldmg(cell_pls), len_mns = dx2(cell_mns), len_pls = dx2(cell_pls) )
ldt(cell_mns::Cell, cell_pls::Cell)  = balance_equation_by_ODE.sum_resistance( val_mns = ldtl(cell_mns) + ldtg(cell_mns), val_pls = ldtl(cell_pls) + ldtg(cell_pls), len_mns = dx2(cell_mns), len_pls = dx2(cell_pls) )

ldt (generic function with 1 method)

## 5.1 多孔質材料中における計算条件

温度、水分化学ポテンシャルの2値を出力する関数  
※2021/07/30 チェック済み

### 5.1.1 温度計算

In [42]:
function cal_newtemp_by_ODE( cell_mns::Cell, cell_cen::Cell, cell_pls::Cell, 
        dt, heat_gain::Float64 );
    val = Porous_material_based_miu
    ODE = balance_equation_by_ODE
    
    r = ODE.latent_heat(val.temp(cell_cen))
    
    # A1の計算
    A1 = ODE.cal_A1( crow = val.crow(cell_cen), vol = val.dx(cell_cen) )
    
    c1 = ODE.cal_c1( lam_mns = lam(cell_mns, cell_cen), lam_pls = lam(cell_cen, cell_pls), ldtg_mns = ldtg(cell_mns, cell_cen), ldtg_pls = ldtg(cell_cen, cell_pls), 
        r = r, dx2_mns = val.dx2(cell_mns), dx2_pls = val.dx2(cell_pls))
    c2 = ODE.cal_c2( temp_mns = val.temp(cell_mns), temp_pls = val.temp(cell_pls), miu_mns = val.miu(cell_mns), miu_pls = val.miu(cell_pls), miu_cen = val.miu(cell_cen), 
        lam_mns = lam(cell_mns, cell_cen), lam_pls = lam(cell_cen, cell_pls), ldmg_mns = ldmg(cell_mns, cell_cen), ldmg_pls = ldmg(cell_cen, cell_pls), ldtg_mns = ldtg(cell_mns, cell_cen), ldtg_pls = ldtg(cell_cen, cell_pls),
        r = r, dx2_mns = val.dx2(cell_mns), dx2_pls = val.dx2(cell_pls))    
    
    return ODE.cal_newtemp( val.temp(cell_cen), A1, c1, c2, dt)
end

cal_newtemp_by_ODE (generic function with 1 method)

### 5.1.2 水分化学ポテンシャル計算

In [43]:
function cal_newmiu_by_ODE( cell_mns::Cell, cell_cen::Cell, cell_pls::Cell, 
        dt, nx::Float64, moisture_gain::Float64 );
    val = Porous_material_based_miu
    ODE = balance_equation_by_ODE
    
    # A2の計算
    A2 = ODE.cal_A2( vol  = val.dx(cell_cen),  dphi = val.dphi(cell_cen) )
    
    c3 = ODE.cal_c3( ldm_mns = ldm(cell_mns, cell_cen), ldm_pls = ldm(cell_cen, cell_pls), dx2_mns = val.dx2(cell_mns), dx2_pls = val.dx2(cell_pls))    
    # ここでは液水の温度勾配水分拡散係数は無視している。
    c4 = ODE.cal_c4( temp_mns = val.temp(cell_mns), temp_pls = val.temp(cell_pls), temp_cen = val.temp(cell_cen), miu_mns = val.miu(cell_mns), miu_pls = val.miu(cell_pls),
        ldm_mns = ldm(cell_mns, cell_cen), ldm_pls = ldm(cell_cen, cell_pls), ldt_mns = ldtg(cell_mns, cell_cen), ldt_pls = ldtg(cell_cen, cell_pls), 
        dx2_mns = val.dx2(cell_mns), dx2_pls = val.dx2(cell_pls), nx = nx )
    
    return ODE.cal_newmiu( val.miu(cell_cen), A2, c3, c4, dt)
end

cal_newmiu_by_ODE (generic function with 1 method)


## 5.2 mns側（先端）が空気の場合の計算条件

### 5.2.1 温度計算

In [44]:
aldt(air)

4.195672904856689e-6

In [45]:
ldtg(cell1) / cell1.dx2

5.282407437407904e-5

In [46]:
# mnsが空気(air)、cenが壁表面(surf)、plsが壁内部(wall)
function cal_newtemp_by_ODE( cell_mns::Air, cell_cen::Cell, cell_pls::Cell, 
        dt, heat_gain::Float64 );
    valw = Porous_material_based_miu
    vala = Air_based_RH
    ODE = balance_equation_by_ODE
    
    r = ODE.latent_heat(valw.temp(cell_cen))
    
    # A2の計算
    A1 = ODE.cal_A1( crow = valw.crow(cell_cen), vol = valw.dx(cell_cen) )
    
    c1 = ODE.cal_c1_BC( 
        lam   = lam(cell_cen, cell_pls), 
        ldtg  = ldtg(cell_cen, cell_pls), 
        alpha = alpha(cell_mns), 
        aldt  = aldt(cell_mns), 
        r = r, 
        dx2 = valw.dx2(cell_cen) + valw.dx2(cell_pls))
    c2 = ODE.cal_c2_BC( 
        temp_wall = valw.temp(cell_pls), temp_air = vala.temp(cell_mns), 
        miu_wall = valw.miu(cell_pls), miu_air = vala.miu(cell_mns), miu_surf = valw.miu(cell_cen), 
        lam   = lam(cell_cen, cell_pls), 
        ldmg  = ldmg(cell_cen, cell_pls), 
        ldtg  = ldtg(cell_cen, cell_pls), 
        alpha = alpha(cell_mns), 
        aldmu = aldmu(cell_mns), 
        aldt  = aldt(cell_mns),
        r = r, 
        dx2 = valw.dx2(cell_cen) + valw.dx2(cell_pls), 
        heat_gain = heat_gain )    
    return ODE.cal_newtemp( valw.temp(cell_cen), A1, c1, c2, dt)
end

cal_newtemp_by_ODE (generic function with 2 methods)

### 5.2.2 水分化学ポテンシャル計算

In [47]:
# mnsが空気(air)、cenが壁表面(surf)、plsが壁内部(wall)
function cal_newmiu_by_ODE( cell_mns::Air, cell_cen::Cell, cell_pls::Cell, 
        dt, nx::Float64, moisture_gain::Float64 );
    valw = Porous_material_based_miu
    vala = Air_based_RH
    ODE = balance_equation_by_ODE
    
    # A2の計算
    A2 = ODE.cal_A2( vol  = valw.dx(cell_cen),  dphi = valw.dphi(cell_cen) )
    
    c3 = ODE.cal_c3_BC(
        ldm   = ldm(cell_cen, cell_pls), 
        aldmu = aldmu(cell_mns), 
        dx2   = valw.dx2(cell_cen) + valw.dx2(cell_pls))
    c4 = ODE.cal_c4_BC(
        temp_wall = valw.temp(cell_pls), temp_air = vala.temp(cell_mns), temp_surf = valw.temp(cell_cen), 
        miu_wall  = valw.miu(cell_pls),  miu_air  = vala.miu(cell_mns),
        ldm       = ldm(cell_cen, cell_pls), 
        ldt       = ldtg(cell_cen, cell_pls), 
        aldmu     = aldmu(cell_mns), 
        aldt      = aldt(cell_mns), 
        dx2       = valw.dx2(cell_cen) + valw.dx2(cell_pls), 
        nx        = nx, 
        moisture_gain = moisture_gain)    
    return ODE.cal_newmiu( valw.miu(cell_cen), A2, c3, c4, dt)
end

cal_newmiu_by_ODE (generic function with 2 methods)

## 5.3 pls側（先端）が空気の場合の計算条件

### 5.3.1 温度計算

In [48]:
# mnsが壁内部(wall)、cenが壁表面(surf)、plsが空気(air)
function cal_newtemp_by_ODE( cell_mns::Cell, cell_cen::Cell, cell_pls::Air, 
        dt, heat_gain::Float64 );
    valw = Porous_material_based_miu
    vala = Air_based_RH
    ODE = balance_equation_by_ODE
    
    r = ODE.latent_heat(valw.temp(cell_cen))
    
    # A1の計算
    A1 = ODE.cal_A1( crow = valw.crow(cell_cen), vol = valw.dx(cell_cen) )
    
    c1 = ODE.cal_c1_BC( 
        lam   = lam(cell_mns, cell_cen), 
        ldtg  = ldtg(cell_mns, cell_cen), 
        alpha = alpha(cell_pls), 
        aldt  = aldt( cell_pls), 
        r     = r, 
        dx2   = valw.dx2(cell_cen) + valw.dx2(cell_mns))
    c2 = ODE.cal_c2_BC( 
        temp_wall = valw.temp(cell_mns), temp_air = vala.temp(cell_pls), 
        miu_wall  = valw.miu(cell_mns),  miu_air  = vala.miu(cell_pls), miu_surf = valw.miu(cell_cen), 
        lam   = lam(cell_mns, cell_cen), 
        ldmg  = ldmg(cell_mns, cell_cen), 
        ldtg  = ldtg(cell_mns, cell_cen), 
        alpha = alpha(cell_pls), 
        aldmu = aldmu(cell_pls), 
        aldt  = aldt(cell_pls),
        r = r, 
        dx2 = valw.dx2(cell_cen) + valw.dx2(cell_mns), 
        heat_gain = heat_gain )
    return ODE.cal_newtemp( valw.temp(cell_cen), A1, c1, c2, dt)
end

cal_newtemp_by_ODE (generic function with 3 methods)

### 5.3.2 水分化学ポテンシャル計算

In [49]:
# mnsが壁内部(wall)、cenが壁表面(surf)、plsが空気(air)
function cal_newmiu_by_ODE( cell_mns::Cell, cell_cen::Cell, cell_pls::Air, 
        dt, nx::Float64, moisture_gain::Float64 );
    valw = Porous_material_based_miu
    vala = Air_based_RH
    ODE = balance_equation_by_ODE
    
    r = ODE.latent_heat(valw.temp(cell_cen))
    
    # A2の計算
    A2 = ODE.cal_A2( vol  = valw.dx(cell_cen),  dphi = valw.dphi(cell_cen) )
    
    c3 = ODE.cal_c3_BC( 
        ldm   = ldm(cell_mns, cell_cen), 
        aldmu = aldmu(cell_pls), 
        dx2   = valw.dx2(cell_cen) + valw.dx2(cell_mns))
    c4 = ODE.cal_c4_BC( 
        temp_wall = valw.temp(cell_mns), temp_air = vala.temp(cell_pls), temp_surf = valw.temp(cell_cen), 
        miu_wall  = valw.miu(cell_mns),  miu_air  = vala.miu(cell_pls),
        ldm   = ldm(cell_mns, cell_cen), 
        ldt   = ldtg(cell_mns, cell_cen), 
        aldmu = aldmu(cell_pls), 
        aldt  = aldt(cell_pls), 
        dx2   = valw.dx2(cell_cen) + valw.dx2(cell_mns), 
        nx    = nx, 
        moisture_gain = moisture_gain)    
    return ODE.cal_newmiu( valw.miu(cell_cen), A2, c3, c4, dt)   
end

cal_newmiu_by_ODE (generic function with 3 methods)

### 補足：キーワード変数として関数を再構築

In [50]:
cal_newvalue_by_ODE(; cell_mns, cell_cen, cell_pls, dt, nx = 0.0, heat_gain = 0.0, moisture_gain = 0.0 ) = cal_newvalue_by_ODE( cell_mns, cell_cen, cell_pls, dt, nx, heat_gain, moisture_gain )
cal_newtemp_by_ODE( ; cell_mns, cell_cen, cell_pls, dt, heat_gain = 0.0 )               = cal_newtemp_by_ODE( cell_mns, cell_cen, cell_pls, dt, heat_gain )
cal_newmiu_by_ODE(  ; cell_mns, cell_cen, cell_pls, dt, nx = 0.0, moisture_gain = 0.0 ) = cal_newmiu_by_ODE(  cell_mns, cell_cen, cell_pls, dt, nx, moisture_gain )

cal_newmiu_by_ODE (generic function with 4 methods)

#### 使用例

In [51]:
air.rh

0.65

In [52]:
cal_newtemp_by_ODE( cell_mns = cell1, cell_cen = cell2, cell_pls = air, dt = 10000000000000.0 )

293.2088456368803

In [53]:
cal_newmiu_by_ODE( cell_mns = cell1, cell_cen = cell2, cell_pls = air, dt = 100000000000000000.0 )

-110.31306332325047